In [ ]:
# only for testing different stuff 
# use main.py for training

In [ ]:
# sample image
import torch

from dataset.utils import pNormalize
from dataset.datasets import sentinel
from model.models import UNET

from torchvision import transforms
from torch.utils.data import DataLoader
q_hi = torch.tensor([2102.0, 1716.0, 1398.0, 4732.0, 2434.42919921875, 3701.759765625, 4519.2177734375, 4857.7734375, 3799.80322265625, 3008.8935546875])
q_lo = torch.tensor([102.0, 159.0, 107.0, 77.0, 106.98081970214844, 79.00384521484375, 86.18966674804688, 70.40167236328125, 50.571197509765625, 36.95356750488281])
        
norm = pNormalize(
                maxPer=q_hi,
                minPer=q_lo
)


# Create experimental dataset and loader, rgb=True for 3 channels (default = False)
test_set = sentinel(root_dir='processed-data/dsen_2_256_new_split', img_transform=norm)
test_loader = DataLoader(test_set, batch_size=10, num_workers=0)

print(len(test_set))

# access one image from dataset
img, label = test_set[0]
#print('image shape:', img.shape)


# or  batch from DataLoader
dataiter = iter(test_loader)
images, labels = dataiter.next()





# define model
model = UNET(in_channels=images.shape[1])
model.load_state_dict(torch.load('to/model_epoch_95.pt',map_location=torch.device('cpu')))
# disable gradients etc. model.train() for training https://medium.com/jun94-devpblog/pytorch-6-model-train-vs-model-eval-no-grad-hyperparameter-tuning-3812c216a3bd
model.eval()

output = model(images)

#pred = torch.argmax(output, dim=1)
     # prediction
pred= torch.nn.functional.softmax(output,dim=1)
pred = torch.argmax(pred,dim=1)


#print('prediction:', pred.shape)
#print('labels:', labels.shape)

print('output:',output.shape)
print('labl', labels.shape)
print('pred:',pred.shape)

In [ ]:
## test plot_sample and plot_batch

from train.metrics import plot_sample, plot_batch

for idx,(images,labels) in enumerate(test_loader):
#dataiter = iter(test_loader)
#images, labels = dataiter.next()
    with torch.no_grad():
        output = model(images)
        pred= torch.nn.functional.softmax(output,dim=1)
        pred = torch.argmax(pred,dim=1)
        plot_batch(pred,labels, images)
        
        for i in range(pred.shape[0]):
            plot_sample(pred[i,:,:],labels[i,:,:], images[i,0:3,:,:]  )
            break
            #plot_pred_lbl_rgb(pred[i,:,:],labels[i,:,:], images[i,0:3,:,:]  )
        if idx==0:
            break

In [ ]:
# run this to make sure tensorboard is working
#!conda list | grep tensorboard
import torch
#!conda install tensorboard -y
from torch.utils.tensorboard import SummaryWriter
print(torch.__version__)

#make sure this executes
SummaryWriter()

In [ ]:
#visualise images in tensorboard
from torchvision.utils import make_grid
from datetime import datetime

dataiter = iter(test_loader)
images, labels = dataiter.next()

# Create a grid from the images and show them
# https://pytorch.org/vision/main/auto_examples/plot_visualization_utils.html#sphx-glr-auto-examples-plot-visualization-utils-py
img_grid = make_grid(images)
#show(img_grid)
#matplotlib_imshow(img_grid, one_channel=False)
#print('  '.join(classes[labels[j]] for j in range(4))
# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/visualise_data')

# Write image data to TensorBoard log dir
writer.add_image('Sat Images', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#  tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

#if on remote
## on remote
##tensorboard --logdir <path> --port 6006
## forward everything on port 6006 on server to 
##ssh -L 16006:127.0.0.1:6006 -i vmexjobb_key.pem exjobbare@vdexjobb.westeurope.cloudapp.azure.com

In [ ]:
#visualizing model in tensorboard

# Again, grab a single mini-batch of images
dataiter = iter(test_loader)
images, labels = dataiter.next()

# add_graph() will trace the sample input through your model,
# and render it as a graph.
writer.add_graph(model, images)
writer.flush()